In [ ]:
# i try with the api but i can't get proper api which give real-time news
# here we don't do extract the data from api and stored it in csv file,  we make it like that is use the data and discard

# this first two code:
# So, how does it know it's fake or real?
# Because it learned the difference from historical data:

# Fake news may use clickbait, exaggerated words, or patterns of misinformation

# Real news typically has more neutral language, structured format, and authentic terms

# TF-IDF captures those subtle differences, and Logistic Regression learns which words or patterns are predictive of fake vs. real.

In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import joblib

# Load datasets
df_fake = pd.read_csv("/content/drive/MyDrive/fake_news_detection/Fake.csv")
df_true = pd.read_csv("/content/drive/MyDrive/fake_news_detection/True.csv")

# Label the data
df_fake['label'] = 0  # fake
df_true['label'] = 1  # real

# Combine and clean
df = pd.concat([df_fake, df_true], ignore_index=True)
df = df[['text', 'label']].dropna()

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    df['text'], df['label'], test_size=0.2, random_state=42
)

# TF-IDF vectorization
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Train model
model = LogisticRegression(max_iter=1000)
model.fit(X_train_vec, y_train)

# Evaluate
preds = model.predict(X_test_vec)
accuracy = accuracy_score(y_test, preds)
print(f"✅ Model accuracy: {accuracy:.2%}")

# Save model and vectorizer
joblib.dump(model, "model.pkl")
joblib.dump(vectorizer, "vectorizer.pkl")
print("✅ model.pkl and vectorizer.pkl have been saved.")


✅ Model accuracy: 98.46%
✅ model.pkl and vectorizer.pkl have been saved.


In [ ]:
# model.pkl — Your trained Logistic Regression model

# vectorizer.pkl — Your fitted TF-IDF vectorizer - TF-IDF helps you figure out which words are important in a document — but not too common.

In [ ]:
# Why It's Used in Fake News Detection:

# Helps identify important keywords in real vs. fake articles

# Transforms human language into something a machine learning model can understand


# Once you've transformed your news articles using TF-IDF,
# you need something to learn patterns and make predictions — that’s where the model comes in.




In [ ]:
# Process Flow:
#  News Text

#  TF-IDF Vectorizer → Numbers

#  Model → Learns patterns from labeled data

#  Predicts: REAL or FAKE

In [21]:
import requests
import joblib
import re
import string

# Load model and vectorizer
model = joblib.load("model.pkl")
vectorizer = joblib.load("vectorizer.pkl")

# Preprocess text
def preprocess(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
    text = re.sub(r"\d+", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Prediction with confidence
def check_news_realtime(news_text):
    clean_text = preprocess(news_text)
    vector = vectorizer.transform([clean_text])
    prediction = model.predict(vector)[0]
    confidence = max(model.predict_proba(vector)[0])
    label = "🚨 FAKE NEWS" if prediction == 0 else "✅ REAL NEWS"
    return f"{label} (Confidence: {confidence:.2%})"

# Fetch full news (description + content)
def fetch_latest_news(api_key, country="us", page_size=5):
    url = f"https://newsapi.org/v2/top-headlines?country={country}&pageSize={page_size}&apiKey={api_key}"
    response = requests.get(url)
    articles = response.json().get("articles", [])
    news_texts = []

    for article in articles:
        title = article.get("title", "")
        description = article.get("description", "")
        content = article.get("content", "")
        full_text = f"{title} {description} {content}".strip()
        if full_text:
            news_texts.append(full_text)

    return news_texts

# --- Main Execution ---
if __name__ == "__main__":
    API_KEY = "ff190a48a285413e921b5a91343dd0cd"

    print("\n📢 Checking real-time headlines using NewsAPI...\n")
    news_list = fetch_latest_news(API_KEY)

    for i, news in enumerate(news_list, 1):
        result = check_news_realtime(news)
        print(f"{i}. {news[:100]}...\n   ➤ {result}\n")  # show first 100 chars

    print("✍️ You can now enter your own news to check (type 'exit' to quit):\n")

    while True:
        user_input = input("Enter news headline or article: ")
        if user_input.lower() == "exit":
            print("👋 Exiting...")
            break
        result = check_news_realtime(user_input)
        print(f"➤ {result}\n")



📢 Checking real-time headlines using NewsAPI...

1. NC court sides with GOP candidate in Supreme Court race - Axios The decision, if it stands, could ov...
   ➤ ✅ REAL NEWS (Confidence: 55.19%)

2. ‘Hands Off!’ protests against Trump and Musk are planned across the US - AP News Opponents of Presid...
   ➤ 🚨 FAKE NEWS (Confidence: 67.12%)

✍️ You can now enter your own news to check (type 'exit' to quit):

Enter news headline or article: SEATTLE/WASHINGTON (Reuters) - President Donald Trump called on the U.S. Postal Service on Friday to charge “much more” to ship packages for Amazon (AMZN.O), picking another fight with an online retail giant he has criticized in the past.     “Why is the United States Post Office, which is losing many billions of dollars a year, while charging Amazon and others so little to deliver their packages, making Amazon richer and the Post Office dumber and poorer? Should be charging MUCH MORE!” Trump wrote on Twitter.  The president’s tweet drew fresh attentio

In [32]:
import pandas as pd
import requests
from datetime import datetime
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import joblib
import os

# -----------------------------
# 1. Preprocessing Function
# -----------------------------
def preprocess(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
    text = re.sub(r"\d+", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

# -----------------------------
# 2. Get Real News from API
# -----------------------------
def get_real_news_from_api(api_key, country="us", page_size=100):
    url = f"https://newsapi.org/v2/top-headlines?country={country}&pageSize={page_size}&apiKey={api_key}"
    response = requests.get(url)
    articles = response.json().get("articles", [])

    news_data = []
    for article in articles:
        full_text = f"{article.get('title', '')} {article.get('description', '')} {article.get('content', '')}".strip()
        if full_text:
            news_data.append({
                "text": full_text,
                "label": 1,  # Real news
                "date": datetime.now().strftime("%Y-%m-%d")
            })
    return pd.DataFrame(news_data)

# -----------------------------
# 3. Load Fake News from CSV
# -----------------------------
def load_fake_news(fake_csv_path):
    df = pd.read_csv(fake_csv_path)
    df = df[['text']].copy()
    df['label'] = 0  # Fake news
    return df

# -----------------------------
# 4. Train and Save Model
# -----------------------------
def train_and_save_model(fake_csv_path, api_key):
    print("🔄 Loading fake news...")
    fake_df = load_fake_news(fake_csv_path)
    print(f"✅ Loaded {len(fake_df)} fake articles.")

    print("🌐 Fetching real news from API...")
    real_df = get_real_news_from_api(api_key)
    print(f"✅ Fetched {len(real_df)} real articles.")

    print("🧹 Preprocessing data...")
    fake_df['text'] = fake_df['text'].apply(preprocess)
    real_df['text'] = real_df['text'].apply(preprocess)

    # Combine and shuffle
    df = pd.concat([real_df, fake_df], ignore_index=True)
    df = df.sample(frac=1).reset_index(drop=True)

    print("🧠 Training model with TF-IDF + Logistic Regression...")
    vectorizer = TfidfVectorizer(max_features=5000)
    X = vectorizer.fit_transform(df['text'])
    y = df['label']

    model = LogisticRegression()
    model.fit(X, y)

    print("💾 Saving model and vectorizer...")
    joblib.dump(model, "model.pkl")
    joblib.dump(vectorizer, "vectorizer.pkl")
    print("✅ Model training complete. Files saved as model.pkl and vectorizer.pkl")

# -----------------------------
# 5. Predict User Input
# -----------------------------
def predict_user_input():
    print("\n📰 Enter a news headline or content to check:")
    user_input = input(">>> ")

    if not os.path.exists("model.pkl") or not os.path.exists("vectorizer.pkl"):
        print("❌ Model files not found. Train the model first.")
        return

    model = joblib.load("model.pkl")
    vectorizer = joblib.load("vectorizer.pkl")

    cleaned_text = preprocess(user_input)
    vector = vectorizer.transform([cleaned_text])
    prediction = model.predict(vector)[0]

    result = "🚨 FAKE NEWS" if prediction == 0 else "✅ REAL NEWS"
    print(f"\nPrediction: {result}")

# -----------------------------
# 6. Run It All
# -----------------------------
if __name__ == "__main__":
    FAKE_CSV_PATH = "/content/drive/MyDrive/fake_news_detection/Fake.csv"  # Update path if needed
    API_KEY = "37eaf5fe6f85b63687fd531a31428cd3"  # Replace with your NewsAPI key

    train_and_save_model(FAKE_CSV_PATH, API_KEY)
    predict_user_input()


🔄 Loading fake news...
✅ Loaded 23481 fake articles.
🌐 Fetching real news from API...
✅ Fetched 32 real articles.
🧹 Preprocessing data...
🧠 Training model with TF-IDF + Logistic Regression...
💾 Saving model and vectorizer...
✅ Model training complete. Files saved as model.pkl and vectorizer.pkl

📰 Enter a news headline or content to check:
>>> exit

Prediction: 🚨 FAKE NEWS


In [35]:
import pandas as pd
import requests
from datetime import datetime
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import joblib
import os

# -----------------------------
# 1. Preprocessing Function
# -----------------------------
def preprocess(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
    text = re.sub(r"\d+", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

# -----------------------------
# 2. Get Real News from MediaStack API
# -----------------------------
def get_real_news_from_api(access_key, limit=100):
    url = f"http://api.mediastack.com/v1/news?access_key={access_key}&languages=en&limit={limit}&sort=published_desc"
    response = requests.get(url)
    articles = response.json().get("data", [])

    news_data = []
    for article in articles:
        full_text = f"{article.get('title', '')} {article.get('description', '')}".strip()
        if full_text:
            news_data.append({
                "text": full_text,
                "label": 1,
                "date": datetime.now().strftime("%Y-%m-%d")
            })
    return pd.DataFrame(news_data)

# -----------------------------
# 3. Load Fake News from CSV
# -----------------------------
def load_fake_news(fake_csv_path):
    df = pd.read_csv(fake_csv_path)
    if 'text' not in df.columns:
        raise KeyError("The CSV must have a 'text' column.")
    df = df[['text']].copy()
    df['label'] = 0
    return df

# -----------------------------
# 4. Train and Save Model
# -----------------------------
def train_and_save_model(fake_csv_path, access_key):
    print("🔄 Loading fake news...")
    fake_df = load_fake_news(fake_csv_path)
    print(f"✅ Loaded {len(fake_df)} fake articles.")

    print("🌐 Fetching real news from MediaStack API...")
    real_df = get_real_news_from_api(access_key)
    print(f"✅ Fetched {len(real_df)} real articles.")

    print("🧹 Preprocessing data...")
    fake_df['text'] = fake_df['text'].apply(preprocess)
    real_df['text'] = real_df['text'].apply(preprocess)

    # Combine and shuffle
    df = pd.concat([real_df, fake_df], ignore_index=True)
    df = df.sample(frac=1).reset_index(drop=True)

    print("🧠 Training model with TF-IDF + Logistic Regression...")
    vectorizer = TfidfVectorizer(max_features=5000)
    X = vectorizer.fit_transform(df['text'])
    y = df['label']

    model = LogisticRegression()
    model.fit(X, y)

    print("💾 Saving model and vectorizer...")
    joblib.dump(model, "model.pkl")
    joblib.dump(vectorizer, "vectorizer.pkl")
    print("✅ Model training complete. Files saved as model.pkl and vectorizer.pkl")

# -----------------------------
# 5. Predict User Input
# -----------------------------
def predict_user_input():
    print("\n📰 Enter a news headline or content to check:")
    user_input = input(">>> ")

    if not os.path.exists("model.pkl") or not os.path.exists("vectorizer.pkl"):
        print("❌ Model files not found. Train the model first.")
        return

    model = joblib.load("model.pkl")
    vectorizer = joblib.load("vectorizer.pkl")

    cleaned_text = preprocess(user_input)
    vector = vectorizer.transform([cleaned_text])
    prediction = model.predict(vector)[0]

    result = "🚨 FAKE NEWS" if prediction == 0 else "✅ REAL NEWS"
    print(f"\nPrediction: {result}")

# -----------------------------
# 6. Run It All
# -----------------------------
if __name__ == "__main__":
    FAKE_CSV_PATH = "/content/drive/MyDrive/fake_news_detection/Fake.csv"  # Update with your file path
    ACCESS_KEY = "A61C9283-D0EA-492B-949B-AA07EE7BBA95"  # 🔑 Replace with your MediaStack API key

    train_and_save_model(FAKE_CSV_PATH, ACCESS_KEY)
    predict_user_input()


🔄 Loading fake news...
✅ Loaded 23481 fake articles.
🌐 Fetching real news from MediaStack API...
✅ Fetched 100 real articles.
🧹 Preprocessing data...
🧠 Training model with TF-IDF + Logistic Regression...
💾 Saving model and vectorizer...
✅ Model training complete. Files saved as model.pkl and vectorizer.pkl

📰 Enter a news headline or content to check:
>>> Noida: An 18-month-old girl, kidnapped earlier this week, was rescued by police on Saturday in a swift operation that led to the arrest of a couple involved in the abduction. The police team was rewarded Rs 25,000 by the deputy commissioner of police (DCP), central Noida, for their prompt action. The accused, identified as Renu (44), a native of Aligarh, and Dinesh (28), from Hathras, had allegedly taken the child from Yusufpur Chak Shahberi without the mother's knowledge. DCP Shakti Mohan Avasthy said, "On March 18, Bisrakh police received a complaint from the child's mother stating that her daughter was taken away by Renu and Dinesh

In [52]:
import pandas as pd
import requests
from datetime import datetime
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import joblib
import os

# -----------------------------
# 1. Preprocessing Function
# -----------------------------
def preprocess(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
    text = re.sub(r"\d+", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

# -----------------------------
# 2. Get Real News from Twingly Livefeed
# -----------------------------
def get_real_news_from_twingly_livefeed(api_key, limit=100):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
    }

    # Step 1: Get starting delivery_id from /status
    try:
        status_url = f"https://data.twingly.net/news/b/livefeed/v1/status?apikey={api_key}"
        status_response = requests.get(status_url, headers=headers)
        status_response.raise_for_status()
        delivery_id = status_response.json()["latest_delivery_id"]
    except Exception as e:
        print(f"❌ Failed to get status: {e}")
        return pd.DataFrame(columns=["text", "label", "date"])

    # Step 2: Fetch real news using delivery_id
    try:
        feed_url = f"https://data.twingly.net/news/b/livefeed/v1/feed?apikey={api_key}&since_id={delivery_id}"
        feed_response = requests.get(feed_url, headers=headers)
        feed_response.raise_for_status()
        articles = feed_response.json().get("posts", [])

        news_data = []
        for article in articles[:limit]:
            full_text = f"{article.get('title', '')} {article.get('summary', '')}".strip()
            if full_text:
                news_data.append({
                    "text": full_text,
                    "label": 1,
                    "date": datetime.now().strftime("%Y-%m-%d")
                })

        return pd.DataFrame(news_data)

    except Exception as e:
        print(f"❌ Failed to fetch from Twingly Livefeed: {e}")
        return pd.DataFrame(columns=["text", "label", "date"])

# -----------------------------
# 3. Load Fake News from CSV
# -----------------------------
def load_fake_news(fake_csv_path):
    df = pd.read_csv(fake_csv_path)
    if 'text' not in df.columns:
        raise KeyError("The CSV must have a 'text' column.")
    df = df[['text']].copy()
    df['label'] = 0
    return df

# -----------------------------
# 4. Train and Save Model
# -----------------------------
def train_and_save_model(fake_csv_path, api_key):
    print("🔄 Loading fake news...")
    fake_df = load_fake_news(fake_csv_path)
    print(f"✅ Loaded {len(fake_df)} fake articles.")

    print("🌐 Fetching real news from Twingly Livefeed...")
    real_df = get_real_news_from_twingly_livefeed(api_key)
    print(f"✅ Fetched {len(real_df)} real articles.")

    if real_df.empty:
        print("⚠️ Real news dataset is empty. Aborting training to avoid bias.")
        return

    print("🧹 Preprocessing data...")
    fake_df['text'] = fake_df['text'].apply(preprocess)
    real_df['text'] = real_df['text'].apply(preprocess)

    df = pd.concat([real_df, fake_df], ignore_index=True)
    df = df.sample(frac=1).reset_index(drop=True)

    print("🧠 Training model with TF-IDF + Logistic Regression...")
    vectorizer = TfidfVectorizer(max_features=5000)
    X = vectorizer.fit_transform(df['text'])
    y = df['label']

    model = LogisticRegression()
    model.fit(X, y)

    print("💾 Saving model and vectorizer...")
    joblib.dump(model, "model.pkl")
    joblib.dump(vectorizer, "vectorizer.pkl")
    print("✅ Model training complete. Files saved as model.pkl and vectorizer.pkl")

# -----------------------------
# 5. Predict User Input
# -----------------------------
def predict_user_input():
    print("\n📰 Enter a news headline or content to check:")
    user_input = input(">>> ")

    if not os.path.exists("model.pkl") or not os.path.exists("vectorizer.pkl"):
        print("❌ Model files not found. Train the model first.")
        return

    model = joblib.load("model.pkl")
    vectorizer = joblib.load("vectorizer.pkl")

    cleaned_text = preprocess(user_input)
    vector = vectorizer.transform([cleaned_text])
    prediction = model.predict(vector)[0]

    result = "🚨 FAKE NEWS" if prediction == 0 else "✅ REAL NEWS"
    print(f"\nPrediction: {result}")

# -----------------------------
# 6. Run Everything
# -----------------------------
if __name__ == "__main__":
    FAKE_CSV_PATH = "/content/drive/MyDrive/fake_news_detection/Fake.csv"  # Update path
    API_KEY = "A61C9283-D0EA-492B-949B-AA07EE7BBA95"  # Your Twingly Livefeed API key

    train_and_save_model(FAKE_CSV_PATH, API_KEY)
    predict_user_input()


🔄 Loading fake news...
✅ Loaded 23481 fake articles.
🌐 Fetching real news from Twingly Livefeed...
❌ Failed to fetch from Twingly Livefeed: 400 Client Error: Bad Request for url: https://data.twingly.net/news/b/livefeed/v1/feed?apikey=A61C9283-D0EA-492B-949B-AA07EE7BBA95&since_id=10138059155
✅ Fetched 0 real articles.
⚠️ Real news dataset is empty. Aborting training to avoid bias.

📰 Enter a news headline or content to check:
>>> exit

Prediction: 🚨 FAKE NEWS
